In [1]:
import os
import h5py
import numpy as np
import scipy
import matplotlib.pyplot as plt


In [2]:
raw_files = sorted(os.listdir("./Z_Data/"))
print(raw_files)

['BPSK_772022_PC_0.hdf5', 'PSK16_772022_PC_0.hdf5', 'PSK8_772022_PC_0.hdf5', 'QAM16_772022_PC_0.hdf5', 'QAM32_772022_PC_0.hdf5', 'QAM64_772022_PC_0.hdf5', 'QPSK_772022_PC_0.hdf5']


In [3]:
classes = set()
for i in range(len(raw_files)):
    classes.add(raw_files[i][:5])
labels = sorted(list(classes))
print(labels)

['BPSK_', 'PSK16', 'PSK8_', 'QAM16', 'QAM32', 'QAM64', 'QPSK_']


In [5]:
with h5py.File('./processed.h5', 'w') as hf:
    data = hf.create_dataset("Data", (1,256, 4), maxshape=(None,256, 4))
    label = hf.create_dataset("Label", (1,1), maxshape=(None,1))
    data[0,:] = np.zeros((256,4), dtype=float)
    label[0] = np.zeros((1), dtype=float)
    print (data.shape)
    print (data[0,:,:])
    
    print (label.shape)
    print (label[0])

(1, 256, 4)
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 ...
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
(1, 1)
[0.]


In [6]:
# random generation between 128 --- 384
print(np.random.randint(0,100))

34


In [7]:
for i in range(len(raw_files)):
    with h5py.File('./Z_Data/'+raw_files[i], 'r') as hf:
        a_group_key = list(hf.keys())
        data = np.array(hf[a_group_key[-1]])            
        print(raw_files[i],a_group_key[-1], len(data))    
        I_data = np.real(data[:])
        Q_data = np.imag(data[:])
        M_data = np.abs(data[:])
        P_data = np.angle(data[:])
        pointer = 5000
        print(I_data.shape,Q_data.shape,M_data.shape,P_data.shape)
        
        #internal loop#
        sub_sections = 1000
        switch = True
        
        while (switch):
            out = np.zeros((sub_sections,256,4), dtype=float)
            for j in range(sub_sections):
                if pointer+256 < len(data):
                    out[j,:,0] = I_data[pointer:pointer+256]
                    out[j,:,1] = Q_data[pointer:pointer+256]
                    out[j,:,2] = M_data[pointer:pointer+256]
                    out[j,:,3] = P_data[pointer:pointer+256]
                    pointer += np.random.randint(128,384)
                else:
                    switch =False
                    break
            if (switch):
                label = np.ones((sub_sections,1), dtype=float)*labels.index(raw_files[i][:5])

                with h5py.File('./processed.h5', 'a') as hf0:
                    hf0["Data"].resize(hf0["Data"].shape[0]+sub_sections, axis = 0)
                    hf0["Data"][-sub_sections:,:,:] = out

                    hf0["Label"].resize(hf0["Label"].shape[0]+sub_sections, axis = 0)
                    hf0["Label"][-sub_sections:] = label    


BPSK_772022_PC_0.hdf5 data 24000000
(24000000,) (24000000,) (24000000,) (24000000,)
PSK16_772022_PC_0.hdf5 data 24000366
(24000366,) (24000366,) (24000366,) (24000366,)
PSK8_772022_PC_0.hdf5 data 24000188
(24000188,) (24000188,) (24000188,) (24000188,)
QAM16_772022_PC_0.hdf5 data 24000292
(24000292,) (24000292,) (24000292,) (24000292,)
QAM32_772022_PC_0.hdf5 data 24000189
(24000189,) (24000189,) (24000189,) (24000189,)
QAM64_772022_PC_0.hdf5 data 23999898
(23999898,) (23999898,) (23999898,) (23999898,)
QPSK_772022_PC_0.hdf5 data 23999822
(23999822,) (23999822,) (23999822,) (23999822,)


In [ ]:
with h5py.File('./processed.h5', 'r') as hf:
    a_group_key = list(hf.keys())
    data = hf[a_group_key[0]]
    label = hf[a_group_key[1]]      

    print(a_group_key)
    print(data.shape, label.shape)
    